In [1]:
# User selection
# Input:
#▪ A large textual file containing a set of records
#▪ Each line contains the information about one single user
#▪ Each line has the format
#  UserId,Name,Surname,Gender,YearOfBirth,City,Education
#▪ A large textual file with pairs (Userid, MovieGenre)
#▪ Each line contains pair Userid, MovieGenre with the format
#  Userid,MovieGenre
#  It means that UserId likes movies of genre MovieGenre


# Output:
#▪ One record for each user that likes both Commedia and Adventure movies
# Each output record contains only Gender and YearOfBirth of a selected user
#▪ Gender,YearOfBirth
#▪ Duplicate pairs must not be removed




In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\Study\\anaconda3\\lib\\site-packages\\pyspark'

In [3]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\Study\Documents\Jupyter Notebooks\PoliTO\1-30 MapReduce Exercises\Exercise-29


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Exercise 29") \
    .getOrCreate()
sc = spark.sparkContext

In [5]:
spark

In [6]:
sc

<SparkContext master=local[*] appName=Exercise 29>

## Reading the users file

In [11]:
rdd29_users_raw = sc.textFile("users.txt").map(lambda uu: uu.split(','))
rdd29_users_raw.take(10)

[['User#1', 'John', 'Smith', 'M', '1934', 'New York', 'Bachelor'],
 ['User#2', 'Paul', 'Jones', 'M', '1956', 'Dallas', 'College'],
 ['User#3', 'Jenny', 'Smith', 'F', '1934', 'Philadelphia', 'Bachelor']]

In [12]:
rdd29_users_proc = rdd29_users_raw.map(lambda uu: (uu[0], uu[1:]) )
rdd29_users_proc.take(10)

[('User#1', ['John', 'Smith', 'M', '1934', 'New York', 'Bachelor']),
 ('User#2', ['Paul', 'Jones', 'M', '1956', 'Dallas', 'College']),
 ('User#3', ['Jenny', 'Smith', 'F', '1934', 'Philadelphia', 'Bachelor'])]

## Reading the likes file

In [10]:
rdd29_likes_raw = sc.textFile("likes.txt").map(lambda uu: uu.split(','))
rdd29_likes_raw.take(10)

[['User#1', 'Commedia'],
 ['User#1', 'Adventure'],
 ['User#1', 'Drama'],
 ['User#2', 'Commedia'],
 ['User#2', 'Crime'],
 ['User#3', 'Commedia'],
 ['User#3', 'Horror'],
 ['User#3', 'Adventure']]

In [14]:
rdd29_likes_proc = rdd29_likes_raw.map(lambda uu: (uu[0], uu[1]) )
rdd29_likes_proc.take(10)

[('User#1', 'Commedia'),
 ('User#1', 'Adventure'),
 ('User#1', 'Drama'),
 ('User#2', 'Commedia'),
 ('User#2', 'Crime'),
 ('User#3', 'Commedia'),
 ('User#3', 'Horror'),
 ('User#3', 'Adventure')]

In [15]:
likes_and_users = rdd29_likes_proc.join(rdd29_users_proc)
likes_and_users.take(10)

[('User#3',
  ('Commedia', ['Jenny', 'Smith', 'F', '1934', 'Philadelphia', 'Bachelor'])),
 ('User#3',
  ('Horror', ['Jenny', 'Smith', 'F', '1934', 'Philadelphia', 'Bachelor'])),
 ('User#3',
  ('Adventure', ['Jenny', 'Smith', 'F', '1934', 'Philadelphia', 'Bachelor'])),
 ('User#1',
  ('Commedia', ['John', 'Smith', 'M', '1934', 'New York', 'Bachelor'])),
 ('User#1',
  ('Adventure', ['John', 'Smith', 'M', '1934', 'New York', 'Bachelor'])),
 ('User#1', ('Drama', ['John', 'Smith', 'M', '1934', 'New York', 'Bachelor'])),
 ('User#2', ('Commedia', ['Paul', 'Jones', 'M', '1956', 'Dallas', 'College'])),
 ('User#2', ('Crime', ['Paul', 'Jones', 'M', '1956', 'Dallas', 'College']))]

qs_and_as

In [38]:
likes_and_users_proc = likes_and_users.map(lambda dd:(dd[0],dd[1][0],dd[1][1][2],dd[1][1][3]) ) 
likes_and_users_proc.take(10)                          

[('User#3', 'Commedia', 'F', '1934'),
 ('User#3', 'Horror', 'F', '1934'),
 ('User#3', 'Adventure', 'F', '1934'),
 ('User#1', 'Commedia', 'M', '1934'),
 ('User#1', 'Adventure', 'M', '1934'),
 ('User#1', 'Drama', 'M', '1934'),
 ('User#2', 'Commedia', 'M', '1956'),
 ('User#2', 'Crime', 'M', '1956')]

In [50]:
likes_and_users_proc = likes_and_users_proc.filter(lambda qw: qw[1] in ('Commedia','Adventure' ) )
likes_and_users_proc.take(10)

[('User#3', 'Commedia', 'F', '1934'),
 ('User#3', 'Adventure', 'F', '1934'),
 ('User#1', 'Commedia', 'M', '1934'),
 ('User#1', 'Adventure', 'M', '1934'),
 ('User#2', 'Commedia', 'M', '1956')]

In [53]:
result = likes_and_users_proc.map(lambda ii: ( ii[0],1) ) 
#result = likes_and_users_proc.map(lambda ii: (ii[0], )
result.take(10)

[('User#3', 1), ('User#3', 1), ('User#1', 1), ('User#1', 1), ('User#2', 1)]

In [58]:
final_result = result.reduceByKey(lambda a,b : a+b).filter(lambda uu: uu[1]==2)
final_result.take(10)

[('User#3', 2), ('User#1', 2)]

In [68]:
final_final_result = final_result.leftOuterJoin(rdd29_users_proc)
final_final_result.take(10)

[('User#1', (2, ['John', 'Smith', 'M', '1934', 'New York', 'Bachelor'])),
 ('User#3', (2, ['Jenny', 'Smith', 'F', '1934', 'Philadelphia', 'Bachelor']))]

In [ ]:
final_final_result = final_final_result.mapValues(lambda yy: yy[0] )
final_final_result.take(10)